# Импорт библиотек

In [1]:
import numpy as np
from gzip import open # NB: overrides standard open()
import pickle as pkl
import pandas as pd
import warnings 
from glob import glob
import os
import math
import functools

### Отключим предупреждения Anaconda

In [2]:
warnings.simplefilter('ignore')

# Загрузим данные

In [3]:
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/y.pkl.gz', 'rb'))

# N-PLS1

## Выберем столбец у с которым будем работать

In [4]:
m=0

# Приближение к правильному алгоритму

In [104]:
from sklearn.model_selection import train_test_split

In [219]:
X_train, X_test, y_train, y_test = train_test_split(
    Xdata_numpy["X"], Ydata.iloc[:,m], test_size=0.2857 
    )

In [220]:
X_train.shape[0]

25

In [279]:
x=X_train

In [280]:
y=y_train

In [281]:
y=np.array(y)

In [282]:
x=np.array(x)

In [264]:
yo=y

In [316]:
# a-numpy array;  b-pandas array
def error(a,b):
    s=0
    for i in range(0,len(a)):
        s+=((a[i]-b.iloc[i]))**2
    return s

In [329]:
def function_1(x,y,yo,fn):
    Tt=[]
    mass=np.zeros([y.shape[0]])
    output=[]
    for f in range(0,fn):
        z=np.zeros([x.shape[1],x.shape[2]])
        for i in range(0,x.shape[2]):
            for j in range(0,x.shape[1]):
                zsum=0
                for k in range(0,x.shape[0]):
                    zsum= zsum+ x[k,j,i]*y[k]
                z[j,i]=zsum
        Wk, S, WI = np.linalg.svd(z)
        w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
        w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
        t=[]
        for h in range(0,x.shape[0]):
            t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
        t=np.array(t).reshape(x.shape[0],1)
        Tt+=[t]
        T=np.array(Tt).reshape(x.shape[0],f+1)
        bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
    
        for g in range(0,x.shape[0]):
            x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
        y=y-(np.dot(T,bf))
        mass+=np.dot(T,bf)
        output+=[error(mass,yo)]
    return output

## function_2 обеспечит v разовое обучение, чтобы уменьшить зависимость от того, как разрезать данные для обучения

In [334]:
    output2=[]
    for p in range(0,1):
        X_train, X_test, y_train, y_test = train_test_split(
            Xdata_numpy["X"], Ydata.iloc[:,m], test_size=0.2857  
            )
        x=X_train
        x=np.array(x)
        y=y_train
        y=np.array(y)
        yo=y_train
        output2+=function_1(x,y,y_train,29)
        print(function_1(x,y,y_train,29))
    print()
    print(output2)

[7458.117246289611, 7142.205052096937, 6116.042564266906, 5415.237073464247, 4324.07352094268, 3276.0374007366936, 2192.447268249104, 1378.872119484022, 1321.3578038752628, 1156.8658405972922, 1057.9871188533853, 968.1390641670208, 843.594329646502, 612.6485336077437, 360.43065737030327, 176.1449317221609, 110.442042767776, 32.44325866287714, 22.790463088046042, 8.989841496611476, 4.246909318785259, 2.234555854630046, 0.33393284054115047, 0.055194666518896485, 0.011781406630588815, 1.4655487401226078e-05, 0.0017195051394770892, 0.0562655795781916, 43.122230359301724]

[6307.553177444917, 5875.046917187505, 5481.456496505078, 4663.845670623172, 4243.206339659942, 3846.6119046429358, 1628.3478871609946, 742.1694020902447, 479.5361522236941, 404.32229340870225, 163.08105454989231, 91.12157138051116, 59.42748855075144, 34.235394972669226, 18.88175594283277, 16.2392630623914, 4.18171028920358, 3.23242569315034, 3.100166419955989, 2.8428465023955023, 2.7673629294661164, 1.5873103270322408, 0

#### Ниже изначальная, но, пока ещё нужная для работы версия function_1

###### Массив mass собирает разложение обратно, и именно он важен для определения числа комронент

In [333]:
Tt=[]
mass=np.zeros([y.shape[0]])
for f in range(0,55):
    z=np.zeros([x.shape[1],x.shape[2]])
    for i in range(0,x.shape[2]):
        for j in range(0,x.shape[1]):
            zsum=0
            for k in range(0,x.shape[0]):
                zsum= zsum+ x[k,j,i]*y[k]
            z[j,i]=zsum
    Wk, S, WI = np.linalg.svd(z)
    w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
    w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
    
    t=[]
    for h in range(0,x.shape[0]):
        t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
    t=np.array(t).reshape(x.shape[0],1)
    Tt+=[t]
    T=np.array(Tt).reshape(x.shape[0],f+1)
    bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
    for g in range(0,x.shape[0]):
        x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
    y=y-(np.dot(T,bf))
    mass+=np.dot(T,bf)
    print(error(mass,y_train))

6594.358996729064
6441.831377203219
6236.7700456701805
4269.057658598826
3672.145891701638
3388.908051426325
2873.6351765110485
2155.5459028393266
1180.6102305201616
715.3067061958991
388.90247591885515
276.8220658151676
152.71341087160997
62.681483649143004
26.1901518889762
7.16061745711831
1.7681931266649182
0.8097579673771593
0.3130067635370711
0.004650589020665965
0.0006584838065105016
0.00023863934639732897
0.00010808572314145802
7.516214311489103e-05
2.0938290779542302e-05
0.00011749920509201098
0.0003283960268211876
0.00021507511304321992
0.00014665573254002727
0.3144115037369245
1.4585239734772737
5.812373079584125
62.74290176150302
307.6321464648081
299.07647877057957
1696.2934876741617
23110.42739214094
116839147.01787284
3233701496.264039
73716608863.07692
157503662986401.78
544323379447697.6
4.76815312348088e+17
1.6618320328065986e+21
9.558141745240109e+24
3.256291471619636e+27
3.348961571918542e+28
8.414818370874985e+29
3.537945350555954e+30
3.5655855260285394e+32
9.928807

# Попробую оформить всё в виде полноценного класса

In [ ]:
class N_PLS1():
    def error(a,b):
        s=0
        for i in range(0,len(a)):
            s+=((a[i]-b.iloc[i]))**2
        return s
    def function_1(x,y,fn):
        Tt=[]
        mass=np.zeros([y.shape[0]])
        output=[]
        for f in range(0,fn):
            z=np.zeros([x.shape[1],x.shape[2]])
            for i in range(0,x.shape[2]):
                for j in range(0,x.shape[1]):
                    zsum=0
                    for k in range(0,x.shape[0]):
                        zsum= zsum+ x[k,j,i]*y[k]
                    z[j,i]=zsum
            Wk, S, WI = np.linalg.svd(z)
            w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
            w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
            t=[]
            for h in range(0,x.shape[0]):
                t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
            t=np.array(t).reshape(x.shape[0],1)
            Tt+=[t]
            T=np.array(Tt).reshape(x.shape[0],f+1)
            bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
            for g in range(0,x.shape[0]):
                x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
            y=y-(np.dot(T,bf))
            mass+=np.dot(T,bf)
            output+=[error(mass,y_train)]
        return output
    def fit(x,y,fn):
        
            
        return y

In [179]:
y

array([34.19308759, 35.72443085, 46.32211944, 35.69579348, 47.44184036,
       33.34204937, 42.44462856, 39.9018207 , 44.39973652, 54.64066407,
       53.71596802, 44.04933373, 36.11255278, 34.24246421, 36.22622795,
       26.1974747 , 41.0398005 , 31.41066637, 34.07047331, 42.14770231,
       45.44679116, 21.41118209, 44.54683093, 35.35460545, 36.47993911])

In [54]:
print(y.shape)

()


In [290]:
function_1(x,y,25)

[41286.08749854996,
 38302.110243945746,
 20444.93670747345,
 13639.701185793923,
 12439.263695246093,
 10789.974224403275,
 8598.56556560382,
 7968.967506273657,
 7064.892691449656,
 6299.901986225843,
 5402.479038865533,
 3654.37066066749,
 3213.2931840440165,
 2682.495895172844,
 1872.8829464517519,
 1219.8609339979532,
 901.9277716135091,
 553.7388536351104,
 252.14526974505077,
 118.10074288670174,
 38.94981464085392,
 21.429430366938657,
 5.790438269180314,
 2.0139509572786625,
 0.8471606395169777]

In [331]:
function_2(Xdata_numpy["X"], Ydata.iloc[:,m], 1, 29, 0.2857)

[5881.081773936589, 5195.194135948176, 4595.753850448796, 3697.4301685787964, 2393.408183981792, 1730.2577746392737, 1137.495276542045, 836.8981056708691, 502.7815170755121, 414.6809716536083, 340.22588862523133, 220.3599620901575, 133.94619309571854, 80.35650253454206, 36.68023612039321, 30.981077033990275, 12.515987829782226, 4.3311267341245285, 1.3863370980748204, 0.7705193007573259, 0.24346041599041418, 0.10774205447739513, 0.016419091188545464, 0.004574090183078223, 0.002686760816077912, 0.0029362552828107406, 0.001584657029826227, 0.002247869747972181, 0.001431492737689517]


[[6389.803909882158,
  5916.599983237266,
  4999.201390296729,
  3737.9558974357187,
  3166.641003683795,
  2791.2273526995987,
  2561.347949798782,
  2155.46758995359,
  1849.5508307278424,
  763.6974357205736,
  628.0873793772564,
  342.44850957395926,
  121.53125255132385,
  46.6384235185164,
  21.163442969920084,
  12.74031224792265,
  10.285472319360716,
  10.064138308163399,
  8.805455535457378,
  8.441746471802308,
  5.545303363744137,
  5.35881305379689,
  1.399291007825839,
  0.03968376890055237,
  0.0006888542845988004,
  9.647630520136338e-09,
  8.177593269649294e-08,
  2.221234637007994e-08,
  8.217963141638592e-06]]